##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: JSON Mode Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates how to use JSON mode.

In [ ]:
!pip install -qU google-generativeai

In [ ]:
import google.generativeai as genai
import json

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Activate JSON mode

Activate JSON mode by specifying `respose_mime_type` in the `generation_config` parameter.

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash-latest",
                              generation_config={"response_mime_type": "application/json"})

An initial, and simpler, approach is to send a prompt guiding how the json response must be, pointing the keys to be used:

In [ ]:
prompt = """Give me a few popular cookie recipes. Consider "recipe_name", "small_description" and "ingredients" as keys in the result."""

In [ ]:
response = model.generate_content(prompt)
print(response.text)

Just for fun, parse the string to JSON, and then serialize it.

In [ ]:
print(json.dumps(json.loads(response.text), indent=4))

Also you can go in a more deterministic approach, where you define spefically the json schema you want to be used:

In [ ]:
prompt = """Give me a few popular cookie recipes. Consider the following json schema for the output:

recipe={ 'recipe_name': str, 'small_description': str, 'ingredients': str}
schema = list[recipe]
"""

In [ ]:
response = model.generate_content(prompt)
print(response.text)

And you can parse this output too:

In [ ]:
print(json.dumps(json.loads(response.text), indent=4))